---

# CSCI 3202, Spring 2024
# Homework 3
# Due: Friday March 8, 2024 at 5:59 PM

<br> 

### Your name: Andrew Logue

<br> 

---

Some useful packages and libraries:



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
import heapq
import unittest
from scipy import stats
import copy as cp
from time import time

---

## Problem 1: Maximizing an Objective Function with a Genetic Algorithm 

Suppose we've lost the index card with our favorite cupcake recipe. We know the ingredients of the cake, but cannot remember the exact amount of each ingredient. We decide to use a genetic algorithm to generate the  ingredient amounts. With each iteration of the genetic algorithm, we bake the cupcakes and taste-test them. We achieve our goal and stop running the genetic algorithm when we get to the actual recipe: 

* 1 tsp salt 
* 3 tsp baking powder 
* 2 cups all-purpose flour 
* 1 cup butter 
* 1 cup granulated sugar 
* 4 large eggs
* 1 tsp vanilla extract
* 1 cup buttermilk 

In [2]:
target = [1, 3, 2, 1, 1, 4, 1, 1]

An example starting state for a member of our population might look like: $state = [1, 2, 100, 36, 60, 3, 5, 50]$

### (1a) 

Write an objective function `def recipe_success(state)` that takes a single argument state, and returns the objective function value (fitness) of the state. The objective function should be maximized when a state reaches the target. You could for example define the fitness score of a particular state based on how far away each entry is from the target recipe.

In [78]:
def recipe_success(state):
    # rather than adjust the 'goodness' for each ingredient, I simply return the difference for each element
    # distance_from_target = [abs(i - j) for i, j in zip(state, target)]
    # print(sum(distance_from_target))
    # fitness = (100/(100 + sum(distance_from_target)))
    fitness = 267
    # fitness += 1*abs(state[0]-1)
    # fitness += 3*abs(state[1]-3)
    # fitness += 96*abs(state[2]-2)
    # fitness += 48*abs(state[3]-1)
    # fitness += 48*abs(state[4]-1)
    # fitness += 3.3*abs(state[5]-4)
    # fitness += 1*abs(state[6]-1)
    # fitness += 48*abs(state[7]-1)
    
    # fitness += 1*state[0]-1
    # fitness += 3*state[1]-3
    # fitness += 96*state[2]-2
    # fitness += 48*state[3]-1
    # fitness += 48*state[4]-1
    # fitness += 3.3*state[5]-4
    # fitness += 1*state[6]-1
    # fitness += 48*state[7]-1
    
    # fitness += state[0]/1-1
    # fitness += state[1]/3-3
    # fitness += state[2]/96-2
    # fitness += state[3]/48-1
    # fitness += state[4]/48-1
    # fitness += state[5]/3.3-4
    # fitness += state[6]/1-1
    # fitness += state[7]/48-1
    
    fitness -= (state[0]-1)/1
    fitness -= (state[1]-3)/3
    fitness -= (state[2]-2)/96
    fitness -= (state[3]-1)/48
    fitness -= (state[4]-1)/48
    fitness -= (state[5]-4)/3.3
    fitness -= (state[6]-1)/1
    fitness -= (state[7]-1)/48
    
    return fitness
    


In [81]:
# Write your own test case to make sure that the target element achieves
# the fitness score you would expect it to (this will vary depending on
# what you did here.)

target = [1, 3, 2, 1, 1, 4, 1, 1]
state = [1, 2, 100, 36, 60, 3, 5, 50]

print("Fitness of state: ", recipe_success(state))
error = [abs(i - j)/i*100 for i, j in zip(target, state)]
print("Error of state: ", error)

# test
state_test1 = [(1 + (j/100))*i for i, j in zip(target, error)]
print("Test state: ", state_test1)
print("Fitness of test state: ", recipe_success(state_test1))

# test
state_test2 = target
print("Test state: ", state_test2)
print("Fitness of test state: ", recipe_success(state_test2))

# test
state_test3 = [100, 100, 100, 100, 100, 100, 100, 100]
print("Test state: ", state_test3)
print("Fitness of test state: ", recipe_success(state_test3))

Fitness of state:  259.6363636363636
Error of state:  [0.0, 33.33333333333333, 4900.0, 3500.0, 5900.0, 25.0, 400.0, 4900.0]
Test state:  [1.0, 4.0, 100.0, 36.0, 60.0, 5.0, 5.0, 50.0]
Fitness of test state:  258.3636363636364
Test state:  [1, 3, 2, 1, 1, 4, 1, 1]
Fitness of test state:  267.0
Test state:  [100, 100, 100, 100, 100, 100, 100, 100]
Fitness of test state:  0.3674242424242209


### (1b) 

Using our in class notebook "Lecture 16 - Genetic Algorithms.ipynb" as your guide, write a genetic algorithm that starts with a population of 100 randomly generated "recipes/states/members" and uses the objective function you wrote in **(1a)** to hopefully hit the target after a certain number of generations. 

Key components of your code:
- Generate the initial population randomly from integers between 0 and 100 
- Allow for mutations in your population with an overall probability of mutation set to p = 0.2
- Choose 2 "parents" in the generation of each "child"
- Choose a random split point at which to combine the two "parents"
- Run the algorithm for 50 iterations ("generations"). Do you hit your target?

In [208]:
class problem:
    
    def __init__(self, initial_population, objective_function, mutation_probability, fitness_goal):
        '''
        initial_population = list of lists; each sub-list is a dna string for a population member
        objective_function = objective function to maximize
        mutation_probability = probability that any given child has a mutation
        fitness_goal = fitness goal to achieve (stopping criterion, once member reaches this)
        '''
        self.population = initial_population
        self.initial_population = initial_population
        self.objective_function = objective_function
        self.p_mutate = mutation_probability
        self.n_pop = len(initial_population)
        self.n_dna = len(initial_population[0])
        self.fitness_goal = fitness_goal

    def fitness(self):
        '''
        calculate each population member's probability of being selected for
        reproduction based on performance on objective function
        '''
        performance = []
        for k in range(self.n_pop):
            performance.append(self.objective_function(self.population[k]))
        total = sum(performance)
        p_reproduce = [perf/sum(performance) for perf in performance]
        return p_reproduce
        
    def reproduce(self, parent1, parent2):
        # last DNA snippet from parent1:
        split = np.random.randint(low=1, high=self.n_dna)
        child = parent1[:split] + parent2[split:]
        return child

    def mutate(self, child):
        # which gene to mutate?
        gene = np.random.randint(low=0, high=self.n_dna)
        if child[gene]==0:
            child[gene]=1
        elif child[gene]==1:
            child[gene]=0
        return child
    
def genetic_algorithm(problem, n_iter):
    
    for t in range(n_iter):
        
        new_generation = []
        
        for k in range(problem.n_pop):
            
            # select for reproduction
            p_reproduce = problem.fitness()
            ind_parents = np.random.choice(range(0,problem.n_pop), size=2, p=p_reproduce, replace=False)
            parent1, parent2 = problem.population[ind_parents[0]], problem.population[ind_parents[1]]
            
            # reproduce
            child = problem.reproduce(parent1, parent2)
            
            # mutate
            l_mutate = np.random.choice([True, False], p=[problem.p_mutate, 1-problem.p_mutate])
            if l_mutate:
                child = problem.mutate(child)
            
            # add to new generation
            new_generation.append(child)
        
        # set problem.population = new generation
        problem.population = new_generation
        
        # exit criterion check
        performance = [problem.objective_function(member) for member in problem.population]
        
        best_member = max(zip(performance, problem.population))
        
        if best_member[0] >= problem.fitness_goal:
            return best_member, t

    # print('reached n_iter')

    return False, False

# Generate the initial population randomly from integers between 0 and 100
initial_population = [np.random.randint(0, 101, size=(len(target))) for i in range(100)]
# Generate the initial population randomly from integers between 0 and 200
# initial_population = [np.random.randint(0, 101, size=(len(target))) for i in range(200)]
initial_population = [i.tolist() for i in initial_population]
# print(initial_population)
# Initialize parameters of genetic problem
genetic_problem = problem(initial_population=initial_population, 
                          fitness_goal=257, 
                          mutation_probability=0.2, # Allow for mutations in your population with an overall probability of mutation set to p = 0.2
                          objective_function=recipe_success)

# Run the algorithm for 50 iterations ("generations").
out, gen = genetic_algorithm(genetic_problem, 50)
print("Best member: ", out)
print("Number of generations: ", gen)
# Did NOT hit target

Best member:  (258.3172348484849, [2, 0, 99, 32, 29, 21, 2, 15])
Number of generations:  12


### (1c)

Report the following:
- How many generations did it take to hit the goal?
- If you change the initial population size to 200, does that change the number of generations it takes to achieve the goal recipe?
- If you change the probability of mutation, does that affect the number of generations it takes to achieve the goal recipe? How so?

Alternate questions to answer if Target not hit:
- Report whether you minimized of maximized the objective function and whether that led to any major changes in how you designed the probability of reproduction. A couple sentences here is fine.
 
- Report how many ingredients you ended up matching. e.g. target = [0.5, 3, 2.5, 1, 1.5, 4, 1, 1.25], perhaps your algorithm achieved [1.5, 3, 8, 1, 1, 100, 56, 1, 1.25], then you would have matched 4 of the ingredient values.
 
- Report how many iterations you tried in order to get this answer. (Don't burn up your machine in the process)

It took an average of approximatley 8.9 generations to hit the goal.

If the initial population size is changed to 200, the average number of generations it takes to achieve the goal INCREASED, to approximatley 18.6 generations.

When I increased the probability of mutation, the average number of generations it takes to achieve the goal INCREASED to approximaltey 26. Alternativley, when I decreased the probability of mutation, the average number of generations it takes to achieve the goal INCREASED GREATLY to approximaltey 32.

In [202]:
n_iter = 20
avg_gen = 0;
for i in range(n_iter):
    initial_population = [np.random.randint(0, 101, size=(len(target))) for i in range(100)]
    initial_population = [i.tolist() for i in initial_population]
    genetic_problem = problem(initial_population=initial_population, 
                          fitness_goal=257, 
                          mutation_probability=0.2,
                          objective_function=recipe_success)
    out, gen = genetic_algorithm(genetic_problem, 50)
    
    if (gen > 0):
        avg_gen += gen
        
print("Average generation to hit goal of 257 with no modifications: ", avg_gen/n_iter)

for i in range(n_iter):
    initial_population = [np.random.randint(0, 101, size=(len(target))) for i in range(200)]
    initial_population = [i.tolist() for i in initial_population]
    genetic_problem = problem(initial_population=initial_population, 
                          fitness_goal=257, 
                          mutation_probability=0.2,
                          objective_function=recipe_success)
    out, gen = genetic_algorithm(genetic_problem, 50)
    
    if (gen > 0):
        avg_gen += gen
        
print("Average generation to hit goal of 257 with initial population size of 200: ", avg_gen/n_iter)

for i in range(n_iter):
    initial_population = [np.random.randint(0, 101, size=(len(target))) for i in range(100)]
    initial_population = [i.tolist() for i in initial_population]
    genetic_problem = problem(initial_population=initial_population, 
                          fitness_goal=257, 
                          mutation_probability=0.4,
                          objective_function=recipe_success)
    out, gen = genetic_algorithm(genetic_problem, 50)
    
    if (gen > 0):
        avg_gen += gen
        
print("Average generation to hit goal of 257 with mutation probability of 0.4: ", avg_gen/n_iter)

for i in range(n_iter):
    initial_population = [np.random.randint(0, 101, size=(len(target))) for i in range(100)]
    initial_population = [i.tolist() for i in initial_population]
    genetic_problem = problem(initial_population=initial_population, 
                          fitness_goal=257, 
                          mutation_probability=0.1,
                          objective_function=recipe_success)
    out, gen = genetic_algorithm(genetic_problem, 50)
    
    if (gen > 0):
        avg_gen += gen
        
print("Average generation to hit goal of 257 with mutation probability of 0.1: ", avg_gen/n_iter)
    

Average generation to hit goal of 257 with no modifications:  8.85
Average generation to hit goal of 257 with initial population size of 200:  18.6
Average generation to hit goal of 257 with mutation probability of 0.4:  26.2
Average generation to hit goal of 257 with mutation probability of 0.1:  32.0
